In [ ]:
import pandas as pd
import numpy as np
import keras 
import tensorflow as tf
from tensorflow.keras.applications import ResNet50


In [ ]:
train=pd.read_csv("../input/petfinder-pawpularity-score/train.csv")
sample=pd.read_csv("../input/petfinder-pawpularity-score/sample_submission.csv")


In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


In [ ]:
directory="../input/petfinder-pawpularity-score/train"

path=[]

for i in train['Id']:
    path.append(f'{directory}/{i}.jpg')
train['path'] = path     

X=train[["path","Pawpularity"]]

In [ ]:
gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split = 0.2,
    #brightness_range = (0.1,1.0),
)

In [ ]:
train_images = gen.flow_from_dataframe(
    dataframe = X,
    x_col = 'path',
    y_col = 'Pawpularity',
    target_size =(180,180),
    color_mode='rgb',
    class_mode = "raw",
    batch_size = 64,
    shuffle= True,
    seed=42,
    subset= 'training',
)


validation_images = gen.flow_from_dataframe(
    dataframe = X,
    x_col = 'path',
    y_col = 'Pawpularity',
    target_size =(180,180),
    color_mode='rgb',
    class_mode = "raw",
    batch_size=64,
    shuffle= True,
    seed=42,
    subset= 'validation',
)

In [ ]:
#from tensorflow.keras.applications.resnet50 import ResNet50
RESNET_WEIGHTS_PATH = '../input/d/xanjay/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

base_model = ResNet50(weights=RESNET_WEIGHTS_PATH, include_top=False)


x = base_model.output
x = keras.layers.GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = keras.layers.Dense(512, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = keras.layers.Dense(1, activation='linear')(x)

# this is the model we will train
model = keras.Model(inputs=base_model.input, outputs=predictions)


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(),
    loss='mse',
    metrics=[tf.keras.metrics.RootMeanSquaredError()])

history = model.fit(
                    train_images,
                    validation_data = validation_images,
                    epochs=20,
                    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True)])

In [ ]:
test_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
    #brightness_range = (0.1,1.0),
)

import os
test_files_df=pd.DataFrame()
test_files_df['file']=os.listdir('../input/petfinder-pawpularity-score/test')

test_images = test_gen.flow_from_dataframe(
    dataframe = test_files_df,
    directory="../input/petfinder-pawpularity-score/test",
    x_col = 'file',
    y_col = None,
    #has_ext=True,
    target_size =(180,180),
    color_mode='rgb',
    class_mode = None,
    batch_size = 64,
    shuffle= True,
    seed=42,
)

In [ ]:
pred=model.predict(test_images)
sample.Pawpularity=pred
sample.to_csv("submission.csv",index=False)

In [ ]:
sample